# Model Performance And Results

**Content**:
1. [Libraries](#libraries)
2. [Gridworld Creation](#gridworld)
3. [Functions](#funct)
4. [Unform Dataset Performance](#uniform)
5. [MLP code](#mlp)
6. [Dataset Manipulation](#manipulation)
7. [Bellman Operator](#bellman)

## 1. Libraries <a id = "libraries"> 

In [1]:
import numpy as np
import random
import sys
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.neural_network import MLPRegressor
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split

## 2. Gridworld Creation <a id = "gridworld">

In [2]:
def randPair(s,e):
    return np.random.randint(s,e), np.random.randint(s,e)

class BoardPiece:

    def __init__(self, name, code, pos):
        self.name = name #name of the piece
        self.code = code #an ASCII character to display on the board
        self.pos = pos #2-tuple e.g. (1,4)

class BoardMask:

    def __init__(self, name, mask, code):
        self.name = name
        self.mask = mask
        self.code = code

    def get_positions(self): #returns tuple of arrays
        return np.nonzero(self.mask)

def zip_positions2d(positions): #positions is tuple of two arrays
    x,y = positions
    return list(zip(x,y))

class GridBoard:

    def __init__(self, size=4):
        self.size = size #Board dimensions, e.g. 4 x 4
        self.components = {} #name : board piece
        self.masks = {}

    def addPiece(self, name, code, pos=(0,0)):
        newPiece = BoardPiece(name, code, pos)
        self.components[name] = newPiece

    #basically a set of boundary elements
    def addMask(self, name, mask, code):
        #mask is a 2D-numpy array with 1s where the boundary elements are
        newMask = BoardMask(name, mask, code)
        self.masks[name] = newMask

    def movePiece(self, name, pos):
        move = True
        for _, mask in self.masks.items():
            if pos in zip_positions2d(mask.get_positions()):
                move = False
        if move:
            self.components[name].pos = pos

    def delPiece(self, name):
        del self.components['name']

    def render(self):
        dtype = '<U2'
        displ_board = np.zeros((self.size, self.size), dtype=dtype)
        displ_board[:] = ' '

        for name, piece in self.components.items():
            displ_board[piece.pos] = piece.code

        for name, mask in self.masks.items():
            displ_board[mask.get_positions()] = mask.code

        return displ_board

    def render_np(self):
        num_pieces = len(self.components) + len(self.masks)
        displ_board = np.zeros((num_pieces, self.size, self.size), dtype=np.uint8)
        layer = 0
        for name, piece in self.components.items():
            pos = (layer,) + piece.pos
            displ_board[pos] = 1
            layer += 1

        for name, mask in self.masks.items():
            x,y = self.masks['boundary'].get_positions()
            z = np.repeat(layer,len(x))
            a = (z,x,y)
            displ_board[a] = 1
            layer += 1
        return displ_board

def addTuple(a,b):
    return tuple([sum(x) for x in zip(a,b)])

In [3]:
class Gridworld:

    def __init__(self, size=4, mode='static'):
        if size >= 4:
            self.board = GridBoard(size=size)
        else:
            print("Minimum board size is 4. Initialized to size 4.")
            self.board = GridBoard(size=4)

        #Add pieces, positions will be updated later
        self.board.addPiece('Player','P',(0,0))
        self.board.addPiece('Goal','+',(0,3))
        self.board.addPiece('Pit','-',(0,3))
        self.board.addPiece('Wall','W',(2,3))

        if mode == 'static':
            self.initGridStatic()
        elif mode == 'player':
            self.initGridPlayer()
        else:
            self.initGridRand()

    #Initialize stationary grid, all items are placed deterministically
    def initGridStatic(self):
        #Setup static pieces
        self.board.components['Player'].pos = randPair(0,self.board.size) #Row, Column
        self.board.components['Goal'].pos = (3,0)
        self.board.components['Pit'].pos = (3,1)
        self.board.components['Wall'].pos = (3,2)
        
        if (not self.validateBoard()):
            #print('Invalid grid. Rebuilding..')
            self.initGridStatic()


    #Check if board is initialized appropriately (no overlapping pieces)
    #also remove impossible-to-win boards
    def validateBoard(self):
        valid = True

        player = self.board.components['Player']
        goal = self.board.components['Goal']
        wall = self.board.components['Wall']
        pit = self.board.components['Pit']

        all_positions = [piece for name,piece in self.board.components.items()]
        all_positions = [player.pos, goal.pos, wall.pos, pit.pos]
        if len(all_positions) > len(set(all_positions)):
            return False

        corners = [(0,0),(0,self.board.size), (self.board.size,0), (self.board.size,self.board.size)]
        #if player is in corner, can it move? if goal is in corner, is it blocked?
        if player.pos in corners or goal.pos in corners:
            val_move_pl = [self.validateMove('Player', addpos) for addpos in [(0,1),(1,0),(-1,0),(0,-1)]]
            val_move_go = [self.validateMove('Goal', addpos) for addpos in [(0,1),(1,0),(-1,0),(0,-1)]]
            if 0 not in val_move_pl or 0 not in val_move_go:
                #print(self.display())
                #print("Invalid board. Re-initializing...")
                valid = False

        return valid

    #Initialize player in random location, but keep wall, goal and pit stationary
    def initGridPlayer(self):
        #height x width x depth (number of pieces)
        self.initGridStatic()
        #place player
        self.board.components['Player'].pos = randPair(0,self.board.size)

        if (not self.validateBoard()):
            #print('Invalid grid. Rebuilding..')
            self.initGridPlayer()

    #Initialize grid so that goal, pit, wall, player are all randomly placed
    def initGridRand(self):
        #height x width x depth (number of pieces)
        self.board.components['Player'].pos = randPair(0,self.board.size)
        self.board.components['Goal'].pos = randPair(0,self.board.size)
        self.board.components['Pit'].pos = randPair(0,self.board.size)
        self.board.components['Wall'].pos = randPair(0,self.board.size)

        if (not self.validateBoard()):
            #print('Invalid grid. Rebuilding..')
            self.initGridRand()

    def validateMove(self, piece, addpos=(0,0)):
        outcome = 0 #0 is valid, 1 invalid, 2 lost game
        pit = self.board.components['Pit'].pos
        wall = self.board.components['Wall'].pos
        new_pos = addTuple(self.board.components[piece].pos, addpos)
        if new_pos == wall:
            outcome = 1 #block move, player can't move to wall
        elif max(new_pos) > (self.board.size-1):    #if outside bounds of board
            outcome = 1
        elif min(new_pos) < 0: #if outside bounds
            outcome = 1
        elif new_pos == pit:
            outcome = 2

        return outcome

    def makeMove(self, action):
        #need to determine what object (if any) is in the new grid spot the player is moving to
        #actions in {u,d,l,r}
        def checkMove(addpos):
            if self.validateMove('Player', addpos) in [0,2]:
                new_pos = addTuple(self.board.components['Player'].pos, addpos)
                self.board.movePiece('Player', new_pos)

        if action == 'u': #up
            checkMove((-1,0))
        elif action == 'd': #down
            checkMove((1,0))
        elif action == 'l': #left
            checkMove((0,-1))
        elif action == 'r': #right
            checkMove((0,1))
        else:
            pass

    def reward(self):
        if (self.board.components['Player'].pos == self.board.components['Pit'].pos):
            return -10
        elif (self.board.components['Player'].pos == self.board.components['Goal'].pos):
            return 10
        else:
            return -1

    def display(self):
        return self.board.render()

In [4]:
action_set = {
    0: 'u',
    1: 'd',
    2: 'l',
    3: 'r',
}

## 3. Functions <a id = 'funct'> 

In [5]:
"""
from2dto1d:
    Args: 
        pos(tuple):2d position of the objects(Player, Wall, Goal, Pit) in the gridworld
    Return:
        return(integer): 1d position of the objects(Player, Wall, Goal, Pit)in the gridworld
""" 
def from2dto1d(pos):
    if pos == '(0, 0)':
        return 0
    if pos == '(0, 1)':
        return 1
    if pos == '(0, 2)':
        return 2
    if pos == '(0, 3)':
        return 3
    if pos == '(1, 0)':
        return 4
    if pos == '(1, 1)':
        return 5
    if pos == '(1, 2)':
        return 6
    if pos == '(1, 3)':
        return 7
    if pos == '(2, 0)':
        return 8
    if pos == '(2, 1)':
        return 9
    if pos == '(2, 2)':
        return 10
    if pos == '(2, 3)':
        return 11
    if pos == '(3, 0)':
        return 12
    if pos == '(3, 1)':
        return 13
    if pos == '(3, 2)':
        return 14
    if pos == '(3, 3)':
        return 15

In [6]:
def from1dto2d(pos):
    if pos == 0:
        return (0, 0)
    if pos == 1:
        return (0, 1)
    if pos == 2:
        return (0, 2)
    if pos == 3:
        return (0, 3)
    if pos == 4:
        return (1, 0)
    if pos == 5:
        return (1, 1)
    if pos == 6:
        return (1, 2)
    if pos == 7:
        return (1, 3)
    if pos == 8:
        return (2, 0)
    if pos == 9:
        return (2, 1)
    if pos == 10:
        return (2, 2)
    if pos == 11:
        return (2, 3)
    if pos == 12:
        return (3, 0)
    if pos == 13:
        return (3, 1)
    if pos == 14:
        return (3, 2)
    if pos == 15:
        return (3, 3)

In [7]:
"""
from_num_to_one_hot_encode:
    Args: 
        num(int): number from 0 to 15 representing the state of the objects(Player, Wall, Goal, Pit) in the gridworld
    Return:
        return(tuple): encoded binary code with the size of 16-bit 
"""
def from_num_to_one_hot_encode(num):
    en0,en1,en2,en3,en4,en5,en6,en7,en8,en9,en10,en11,en12,en13,en14,en15 = 0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
    if num == 0:
        en0 = 1
    if num == 1:
        en1 = 1
    if num == 2:
        en2 = 1
    if num == 3:
        en3 = 1
    if num == 4:
        en4 = 1
    if num == 5:
        en5 = 1
    if num == 6:
        en6 = 1
    if num == 7:
        en7 = 1
    if num == 8:
        en8 = 1
    if num == 9:
        en9 = 1
    if num == 10:
        en10 = 1
    if num == 11:
        en11 = 1
    if num == 12:
        en12 = 1
    if num == 13:
        en13 = 1
    if num == 14:
        en14 = 1
    if num == 15:
        en15 = 1
    return en0,en1,en2,en3,en4,en5,en6,en7,en8,en9,en10,en11,en12,en13,en14,en15

In [8]:
def calculate_smape(actual, predicted) -> float:
    if not all([isinstance(actual, np.ndarray), 
                isinstance(predicted, np.ndarray)]):
        actual, predicted = np.array(actual),
        np.array(predicted)
  
    return round(
        np.mean(
            np.abs(predicted - actual) / 
            ((np.abs(predicted) + np.abs(actual))/2)
        )*100, 2
    )

## 4. Unform Dataset MLP Performance <a id = "mlpuniform"> 

In [53]:
df = pd.read_csv("DatasetAfterUniform.csv")
df

,Unnamed: 0,Player,Goal,Wall,Pit,Q_Current,Reward_Current,Bellman_Operator
0,0,0,2,1,0,0.305944,-10,1.284265
1,1,2,2,1,0,9.047926,10,10.000000
2,2,3,2,1,0,7.594888,-1,10.000000
3,3,4,2,1,0,2.538072,-1,4.271501
4,4,5,2,1,0,5.857223,-1,5.000492
...,...,...,...,...,...,...,...,...
50395,50395,8,13,14,15,5.306869,-1,5.827087
50396,50396,13,13,14,15,12.008292,10,10.000000
50397,50397,9,13,14,15,7.040803,-1,10.000000
50398,50398,12,13,14,15,7.585652,-1,10.000000


### Q value approximation

In [54]:
from sklearn.model_selection import train_test_split
X = df[['Player','Pit', 'Goal', 'Wall']]
y = df[['Q_Current']]

In [55]:
mlp = MLPRegressor(hidden_layer_sizes=(200,120), max_iter=300, activation = 'relu', solver='adam', random_state=123)

In [56]:
mlp.fit(X.values,y.values.ravel())

MLPRegressor(hidden_layer_sizes=(200, 120), max_iter=300, random_state=123)

In [57]:
h = 0
w, h = 13, 300000
experience_mlp = [[0 for x in range(w)] for y in range(h)] 

global counter
counter = 0

In [13]:
def mlp_test_model(model_b, experience, mode='static', display=True):
    global counter
    i = 0
    test_game = Gridworld(mode=mode)
    if display:
        print("Initial State:")
        print(test_game.display())
    status = 1
    j = 0
    while(status == 1): #A
        p_curr = from2dto1d((str(test_game.board.components['Player'].pos)))
        pi = from2dto1d((str(test_game.board.components['Pit'].pos)))
        g = from2dto1d((str(test_game.board.components['Goal'].pos)))
        w = from2dto1d((str(test_game.board.components['Wall'].pos)))

        q_value_next = []

        test_game.makeMove('u')
        p = from2dto1d((str(test_game.board.components['Player'].pos)))
        
        if p_curr != p:
            en0,en1,en2,en3,en4,en5,en6,en7,en8,en9,en10,en11,en12,en13,en14,en15 = from_num_to_one_hot_encode(p)
            reward = test_game.reward()
            q_value_next.append(model_b.predict([[p,pi,g,w]])[0])
            test_game.makeMove('d')
        else:
            q_value_next.append(-10)
        
        test_game.makeMove('d')
        p = from2dto1d((str(test_game.board.components['Player'].pos)))
        
        if p_curr != p:
            en0,en1,en2,en3,en4,en5,en6,en7,en8,en9,en10,en11,en12,en13,en14,en15 = from_num_to_one_hot_encode(p)
            reward = test_game.reward()
            q_value_next.append(model_b.predict([[p,pi,g,w]])[0])
            test_game.makeMove('u')
        else:
            q_value_next.append(-10)
        
        test_game.makeMove('l')
        p = from2dto1d((str(test_game.board.components['Player'].pos)))
        

        if p_curr != p:
            test_game.makeMove('r')
            en0,en1,en2,en3,en4,en5,en6,en7,en8,en9,en10,en11,en12,en13,en14,en15 = from_num_to_one_hot_encode(p)
            reward = test_game.reward()
            q_value_next.append(model_b.predict([[p,pi,g,w]])[0])
        else:
            q_value_next.append(-10)
        
        test_game.makeMove('r')
        p = from2dto1d((str(test_game.board.components['Player'].pos)))
        

        if p_curr != p:
            en0,en1,en2,en3,en4,en5,en6,en7,en8,en9,en10,en11,en12,en13,en14,en15 = from_num_to_one_hot_encode(p)
            reward = test_game.reward()
            q_value_next.append(model_b.predict([[p,pi,g,w]])[0])
            test_game.makeMove('l')
        else:
            q_value_next.append(-10)
        
#         print(q_value_next)
        
        
        action_ = np.argmax(q_value_next)
#         print(action_)
        action = action_set[action_]
        
        experience[counter][0] = p_curr
        experience[counter][1] = pi
        experience[counter][2] = g
        experience[counter][3] = w
        experience[counter][4] = action_
        
    
        if display:
            print('Move #: %s; Taking action: %s' % (i, action))
        test_game.makeMove(action)
        p_next = from2dto1d((str(test_game.board.components['Player'].pos)))
        experience[counter][5] = p_next
        
        j += 1
        if display:
            print(test_game.display())
        reward = test_game.reward()
        experience[counter][6] = reward
        if reward > -1:
            experience[counter][7] = True
        else:
            experience[counter][7] = False
        experience[counter][8] =  q_value_next[action_]
        experience[counter][9] = q_value_next[0]
        experience[counter][10] = q_value_next[1]
        experience[counter][11] = q_value_next[2]
        experience[counter][12] = q_value_next[3]
        if reward != -1:
            if reward > 0:
                status = 2
                if display:
                    print("Game won! Reward: %s" % (reward,))
            else:
                status = 0
                if display:
                    print("Game LOST. Reward: %s" % (reward,))
        counter += 1
        i += 1
        if (i > 15):
            if display:
                print("Game lost; too many moves.")
            break
    
    win = True if status == 2 else False
    return win

In [59]:
win_num = 0
for i in range(0,10):
    max_games = 1000
    wins = 0
    for i in range(max_games):
        win = mlp_test_model(mlp, experience_mlp, 'random', display = False)
        if win:
            wins += 1
    win_perc = float(wins) / float(max_games)
    win_num = win_num + win_perc
    print("Games played: {0}, # of wins: {1}".format(max_games,wins))
    print(win_perc)
    print(win_num)
win_num = win_num / 10
print("Win percentage: {}%".format(win_num*100))

Games played: 1000, # of wins: 927
0.927
0.927
Games played: 1000, # of wins: 936
0.936
1.863
Games played: 1000, # of wins: 917
0.917
2.7800000000000002
Games played: 1000, # of wins: 937
0.937
3.7170000000000005
Games played: 1000, # of wins: 928
0.928
4.6450000000000005
Games played: 1000, # of wins: 922
0.922
5.567
Games played: 1000, # of wins: 937
0.937
6.5040000000000004
Games played: 1000, # of wins: 915
0.915
7.4190000000000005
Games played: 1000, # of wins: 935
0.935
8.354000000000001
Games played: 1000, # of wins: 915
0.915
9.269000000000002
Win percentage: 92.69000000000001%


In [60]:
y_pred = mlp.predict(X.values)
calculate_smape(y.values.ravel(),y_pred)

33.6

### Bellman Operator Approximation

In [61]:
from sklearn.model_selection import train_test_split
X = df[['Player','Pit', 'Goal', 'Wall']]
y = df[['Bellman_Operator']]

In [62]:
mlp = MLPRegressor(hidden_layer_sizes=(200,120), max_iter=300, activation = 'relu', solver='adam', random_state=123)

In [63]:
mlp.fit(X.values,y.values.ravel())

MLPRegressor(hidden_layer_sizes=(200, 120), max_iter=300, random_state=123)

In [64]:
h = 0
w, h = 13, 300000
experience_mlp = [[0 for x in range(w)] for y in range(h)] 

global counter
counter = 0

In [65]:
win_num = 0
for i in range(0,10):
    max_games = 1000
    wins = 0
    for i in range(max_games):
        win = mlp_test_model(mlp, experience_mlp, 'random', display = False)
        if win:
            wins += 1
    win_perc = float(wins) / float(max_games)
    win_num = win_num + win_perc
    print("Games played: {0}, # of wins: {1}".format(max_games,wins))
    print(win_perc)
    print(win_num)
win_num = win_num / 10
print("Win percentage: {}%".format(win_num*100))

Games played: 1000, # of wins: 863
0.863
0.863
Games played: 1000, # of wins: 872
0.872
1.7349999999999999
Games played: 1000, # of wins: 889
0.889
2.6239999999999997
Games played: 1000, # of wins: 891
0.891
3.5149999999999997
Games played: 1000, # of wins: 895
0.895
4.41
Games played: 1000, # of wins: 880
0.88
5.29
Games played: 1000, # of wins: 885
0.885
6.175
Games played: 1000, # of wins: 897
0.897
7.072
Games played: 1000, # of wins: 888
0.888
7.96
Games played: 1000, # of wins: 885
0.885
8.845
Win percentage: 88.45%


In [66]:
y_pred = mlp.predict(X.values)
calculate_smape(y.values.ravel(),y_pred)

25.14

## 5. Non-Unform Dataset MLP Performance <a id = "mlpnonuniform"> 

In [80]:
df = pd.read_csv("DatasetAfterNonUniform.csv")
df

,Unnamed: 0,Player,Goal,Wall,Pit,Q_Current,Reward_Current,Bellman_Operator
0,0,2,11,15,8,4.941091,-1.0,4.318901
1,1,6,11,15,8,7.417938,-1.0,7.252085
2,2,7,11,15,8,9.168983,-1.0,10.000000
3,3,6,12,5,13,3.616126,-1.0,1.673050
4,4,10,12,5,13,3.241566,-1.0,5.395397
...,...,...,...,...,...,...,...,...
8758,8758,5,13,8,4,7.248434,-1.0,6.309692
8759,8759,9,13,8,4,8.121880,-1.0,10.000000
8760,8760,6,10,7,1,8.211661,-1.0,10.000000
8761,8761,8,13,1,12,7.724490,-1.0,5.556013


### Q value approximation

In [81]:
from sklearn.model_selection import train_test_split
X = df[['Player','Pit', 'Goal', 'Wall']]
y = df[['Q_Current']]

In [82]:
mlp = MLPRegressor(hidden_layer_sizes=(200,120), max_iter=300, activation = 'relu', solver='adam', random_state=123)

In [83]:
mlp.fit(X.values,y.values.ravel())

MLPRegressor(hidden_layer_sizes=(200, 120), max_iter=300, random_state=123)

In [84]:
h = 0
w, h = 13, 300000
experience_mlp = [[0 for x in range(w)] for y in range(h)] 

global counter
counter = 0

In [85]:
win_num = 0
for i in range(0,10):
    max_games = 1000
    wins = 0
    for i in range(max_games):
        win = mlp_test_model(mlp, experience_mlp, 'random', display = False)
        if win:
            wins += 1
    win_perc = float(wins) / float(max_games)
    win_num = win_num + win_perc
    print("Games played: {0}, # of wins: {1}".format(max_games,wins))
    print(win_perc)
    print(win_num)
win_num = win_num / 10
print("Win percentage: {}%".format(win_num*100))

Games played: 1000, # of wins: 460
0.46
0.46
Games played: 1000, # of wins: 437
0.437
0.897
Games played: 1000, # of wins: 465
0.465
1.362
Games played: 1000, # of wins: 482
0.482
1.844
Games played: 1000, # of wins: 438
0.438
2.282
Games played: 1000, # of wins: 492
0.492
2.774
Games played: 1000, # of wins: 458
0.458
3.232
Games played: 1000, # of wins: 438
0.438
3.6700000000000004
Games played: 1000, # of wins: 455
0.455
4.125
Games played: 1000, # of wins: 454
0.454
4.579
Win percentage: 45.79%


In [86]:
y_pred = mlp.predict(X.values)
calculate_smape(y.values.ravel(),y_pred)

31.66

### Bellman Operator Approximation

In [87]:
from sklearn.model_selection import train_test_split
X = df[['Player','Pit', 'Goal', 'Wall']]
y = df[['Bellman_Operator']]

In [88]:
mlp = MLPRegressor(hidden_layer_sizes=(200,120), max_iter=300, activation = 'relu', solver='adam', random_state=123)

In [89]:
mlp.fit(X.values,y.values.ravel())

MLPRegressor(hidden_layer_sizes=(200, 120), max_iter=300, random_state=123)

In [90]:
h = 0
w, h = 13, 300000
experience_mlp = [[0 for x in range(w)] for y in range(h)] 

global counter
counter = 0

In [91]:
win_num = 0
for i in range(0,10):
    max_games = 1000
    wins = 0
    for i in range(max_games):
        win = mlp_test_model(mlp, experience_mlp, 'random', display = False)
        if win:
            wins += 1
    win_perc = float(wins) / float(max_games)
    win_num = win_num + win_perc
    print("Games played: {0}, # of wins: {1}".format(max_games,wins))
    print(win_perc)
    print(win_num)
win_num = win_num / 10
print("Win percentage: {}%".format(win_num*100))

Games played: 1000, # of wins: 756
0.756
0.756
Games played: 1000, # of wins: 752
0.752
1.508
Games played: 1000, # of wins: 750
0.75
2.258
Games played: 1000, # of wins: 761
0.761
3.019
Games played: 1000, # of wins: 757
0.757
3.7760000000000002
Games played: 1000, # of wins: 758
0.758
4.534000000000001
Games played: 1000, # of wins: 767
0.767
5.301000000000001
Games played: 1000, # of wins: 755
0.755
6.056000000000001
Games played: 1000, # of wins: 765
0.765
6.821000000000001
Games played: 1000, # of wins: 741
0.741
7.562
Win percentage: 75.62%


In [92]:
y_pred = mlp.predict(X.values)
calculate_smape(y.values.ravel(),y_pred)

26.59

## 6. Unform Dataset LR Performance <a id = "lruniform"> 

In [9]:
df = pd.read_csv("DatasetAfterUniform.csv")
df

,Unnamed: 0,Player,Goal,Wall,Pit,Q_Current,Reward_Current,Bellman_Operator
0,0,0,2,1,0,0.305944,-10,1.284265
1,1,2,2,1,0,9.047926,10,10.000000
2,2,3,2,1,0,7.594888,-1,10.000000
3,3,4,2,1,0,2.538072,-1,4.271501
4,4,5,2,1,0,5.857223,-1,5.000492
...,...,...,...,...,...,...,...,...
50395,50395,8,13,14,15,5.306869,-1,5.827087
50396,50396,13,13,14,15,12.008292,10,10.000000
50397,50397,9,13,14,15,7.040803,-1,10.000000
50398,50398,12,13,14,15,7.585652,-1,10.000000


### Q value approximation

In [55]:
X = df[['Player', 'Pit', 'Goal', 'Wall']]
y = df[['Q_Current']]

In [57]:
from sklearn.linear_model import LinearRegression
regressor1 = LinearRegression()
regressor1.fit(X.values,y.values.ravel())

LinearRegression()

In [59]:
y_pred = regressor1.predict(X.values)
calculate_smape(y.values.ravel(),y_pred)

55.63

In [60]:
h = 0
w, h = 13, 300000
experience_mlp = [[0 for x in range(w)] for y in range(h)] 

global counter
counter = 0

In [61]:
win_num = 0
for i in range(0,10):
    max_games = 1000
    wins = 0
    for i in range(max_games):
        win = mlp_test_model(regressor1, experience_mlp, 'random', display = False)
        if win:
            wins += 1
    win_perc = float(wins) / float(max_games)
    win_num = win_num + win_perc
    print("Games played: {0}, # of wins: {1}".format(max_games,wins))
    print(win_perc)
    print(win_num)
win_num = win_num / 10
print("Win percentage: {}%".format(win_num*100))

Games played: 1000, # of wins: 200
0.2
0.2
Games played: 1000, # of wins: 219
0.219
0.41900000000000004
Games played: 1000, # of wins: 204
0.204
0.623
Games played: 1000, # of wins: 210
0.21
0.833
Games played: 1000, # of wins: 198
0.198
1.031
Games played: 1000, # of wins: 199
0.199
1.23
Games played: 1000, # of wins: 204
0.204
1.434
Games played: 1000, # of wins: 199
0.199
1.633
Games played: 1000, # of wins: 183
0.183
1.816
Games played: 1000, # of wins: 217
0.217
2.033
Win percentage: 20.33%


### Bellman Operator Approximation

In [62]:
X = df[['Player', 'Pit', 'Goal', 'Wall']]
y = df[['Bellman_Operator']]

In [63]:
from sklearn.linear_model import LinearRegression
regressor1 = LinearRegression()
regressor1.fit(X.values,y.values.ravel())

LinearRegression()

In [64]:
y_pred = regressor1.predict(X.values)
calculate_smape(y.values.ravel(),y_pred)

50.16

In [65]:
h = 0
w, h = 13, 300000
experience_mlp = [[0 for x in range(w)] for y in range(h)] 

global counter
counter = 0

In [66]:
win_num = 0
for i in range(0,10):
    max_games = 1000
    wins = 0
    for i in range(max_games):
        win = mlp_test_model(regressor1, experience_mlp, 'random', display = False)
        if win:
            wins += 1
    win_perc = float(wins) / float(max_games)
    win_num = win_num + win_perc
    print("Games played: {0}, # of wins: {1}".format(max_games,wins))
    print(win_perc)
    print(win_num)
win_num = win_num / 10
print("Win percentage: {}%".format(win_num*100))

Games played: 1000, # of wins: 202
0.202
0.202
Games played: 1000, # of wins: 211
0.211
0.41300000000000003
Games played: 1000, # of wins: 214
0.214
0.627
Games played: 1000, # of wins: 218
0.218
0.845
Games played: 1000, # of wins: 201
0.201
1.046
Games played: 1000, # of wins: 190
0.19
1.236
Games played: 1000, # of wins: 191
0.191
1.427
Games played: 1000, # of wins: 197
0.197
1.624
Games played: 1000, # of wins: 189
0.189
1.8130000000000002
Games played: 1000, # of wins: 162
0.162
1.975
Win percentage: 19.75%


## 7. Non Unform Dataset LR Performance <a id = "lrnonuniform"> 

In [67]:
df = pd.read_csv("DatasetAfterNonUniform.csv")
df

,Unnamed: 0,Player,Goal,Wall,Pit,Q_Current,Reward_Current,Bellman_Operator
0,0,2,11,15,8,4.941091,-1.0,4.318901
1,1,6,11,15,8,7.417938,-1.0,7.252085
2,2,7,11,15,8,9.168983,-1.0,10.000000
3,3,6,12,5,13,3.616126,-1.0,1.673050
4,4,10,12,5,13,3.241566,-1.0,5.395397
...,...,...,...,...,...,...,...,...
8758,8758,5,13,8,4,7.248434,-1.0,6.309692
8759,8759,9,13,8,4,8.121880,-1.0,10.000000
8760,8760,6,10,7,1,8.211661,-1.0,10.000000
8761,8761,8,13,1,12,7.724490,-1.0,5.556013


In [68]:
X = df[['Player', 'Pit', 'Goal', 'Wall']]
y = df[['Q_Current']]

In [69]:
from sklearn.linear_model import LinearRegression
regressor1 = LinearRegression()
regressor1.fit(X.values,y.values.ravel())

LinearRegression()

In [70]:
y_pred = regressor1.predict(X.values)
calculate_smape(y.values.ravel(),y_pred)

37.81

In [71]:
h = 0
w, h = 13, 300000
experience_mlp = [[0 for x in range(w)] for y in range(h)] 

global counter
counter = 0

In [72]:
win_num = 0
for i in range(0,10):
    max_games = 1000
    wins = 0
    for i in range(max_games):
        win = mlp_test_model(regressor1, experience_mlp, 'random', display = False)
        if win:
            wins += 1
    win_perc = float(wins) / float(max_games)
    win_num = win_num + win_perc
    print("Games played: {0}, # of wins: {1}".format(max_games,wins))
    print(win_perc)
    print(win_num)
win_num = win_num / 10
print("Win percentage: {}%".format(win_num*100))

Games played: 1000, # of wins: 203
0.203
0.203
Games played: 1000, # of wins: 184
0.184
0.387
Games played: 1000, # of wins: 199
0.199
0.5860000000000001
Games played: 1000, # of wins: 202
0.202
0.788
Games played: 1000, # of wins: 210
0.21
0.998
Games played: 1000, # of wins: 213
0.213
1.211
Games played: 1000, # of wins: 198
0.198
1.409
Games played: 1000, # of wins: 200
0.2
1.609
Games played: 1000, # of wins: 220
0.22
1.829
Games played: 1000, # of wins: 192
0.192
2.021
Win percentage: 20.21%


In [73]:
X = df[['Player', 'Pit', 'Goal', 'Wall']]
y = df[['Bellman_Operator']]

In [74]:
from sklearn.linear_model import LinearRegression
regressor1 = LinearRegression()
regressor1.fit(X.values,y.values.ravel())

LinearRegression()

In [75]:
y_pred = regressor1.predict(X.values)
calculate_smape(y.values.ravel(),y_pred)

41.75

In [76]:
h = 0
w, h = 13, 300000
experience_mlp = [[0 for x in range(w)] for y in range(h)] 

global counter
counter = 0

In [77]:
win_num = 0
for i in range(0,10):
    max_games = 1000
    wins = 0
    for i in range(max_games):
        win = mlp_test_model(regressor1, experience_mlp, 'random', display = False)
        if win:
            wins += 1
    win_perc = float(wins) / float(max_games)
    win_num = win_num + win_perc
    print("Games played: {0}, # of wins: {1}".format(max_games,wins))
    print(win_perc)
    print(win_num)
win_num = win_num / 10
print("Win percentage: {}%".format(win_num*100))

Games played: 1000, # of wins: 199
0.199
0.199
Games played: 1000, # of wins: 195
0.195
0.394
Games played: 1000, # of wins: 214
0.214
0.608
Games played: 1000, # of wins: 202
0.202
0.81
Games played: 1000, # of wins: 212
0.212
1.022
Games played: 1000, # of wins: 222
0.222
1.244
Games played: 1000, # of wins: 201
0.201
1.445
Games played: 1000, # of wins: 204
0.204
1.649
Games played: 1000, # of wins: 211
0.211
1.86
Games played: 1000, # of wins: 165
0.165
2.025
Win percentage: 20.25%
